# Generating Biomass, Substrate And Product Fluxes from mcPECASO Object #

In [2]:
import numpy as np
import cameo
import pandas as pd
from copy import deepcopy
import pickle
import time
import multiprocessing
from joblib import Parallel, delayed
from mcpecaso.plotting import multiplot_envelopes,plot_envelope,plot_pecaso_dfba,\
                              multi_two_stage_char_contours,two_stage_char_contour
from mcpecaso.core import mcPECASO
import mcpecaso as mcpecaso
from mcpecaso.core.substrate_dependent_envelopes import linear_uptake, logistic_uptake


mcpecaso.settings.time_end = 100
mcpecaso.settings.initial_biomass = 0.05
mcpecaso.settings.initial_substrate = 500
mcpecaso.settings.num_timepoints = 10000
mcpecaso.settings.scope = 'extrema'

pd.options.display.max_rows = 100

num_cores = multiprocessing.cpu_count()
model = cameo.models.bigg.iJO1366

In [8]:
metabolite_names_dict = {'Spermidine': 'Spermidine',
                         'L-Tryptophan': 'L-Tryptophan',
                         'Thymidine C10H14N2O5': 'Thymidine',
                         'L-Phenylalanine': 'L-Phenylalanine',
                         'Adenosine': 'Adenosine',
                         'Indole': 'Indole',
                         'L-Tyrosine': 'L-Tyrosine',
                         'Inosine': 'Inosine',
                         'Xanthosine': 'Xanthosine',
                         'Cytidine': 'Cytidine',
                         'Uridine': 'Uridine',
                         'Quinate': 'Quinate',
                         'L-Isoleucine': 'L-Isoleucine',
                         'L-Lysine': 'L-Lysine',
                         'Hexanoate (n-C6:0)': 'Hexanoate',
                         'L-Leucine': 'L-Leucine',
                         'L-Arginine': 'L-Arginine',
                         'L-Histidine': 'L-Histidine',
                         'D-Gluconate': 'D-Gluconic Acid',
                         'L-Idonate': 'L-Idonic Acid',
                         '5-Dehydro-D-gluconate': '5-Ketogluconate',
                         'Citrate': 'Citric Acid',
                         'Agmatine': 'Agmatine',
                         'Ornithine': 'Ornithine',
                         'L-Proline': 'L-Proline',
                         'L-Valine': 'L-Valine',
                         'Thymine C5H6N2O2': 'Thymine',
                         'Adenine': 'Adenine',
                         'Guanine': 'Guanine',
                         'Hypoxanthine': 'Hypoxanthine',
                         'O-Acetyl-L-serine': 'O-Acetyl-L-serine',
                         'Xanthine': 'Xanthine',
                         'L-Glutamate': 'L-Glutamate',
                         '2-Oxoglutarate': '\u03b1-Ketoglutarate',
                         'Putrescine': 'Putrescine',
                         'L-Threonine': 'L-Threonine',
                         '4-Aminobutanoate': '\u03b3-Aminobutyrate',
                         'L-Homoserine': 'L-Homoserine',
                         'Allantoin': 'Allantoin',
                         'Succinate': 'Succinate',
                         'Uracil': 'Uracil',
                         'L-Asparagine': 'L-Asparagine',
                         'L-Malate': 'L-Malate',
                         'L-Aspartate': 'L-Aspartate',
                         'L-Cysteine': 'L-Cysteine',
                         'Glycerol 3-phosphate': 'Glycerol3-phosphate',
                         '3-Hydroxypropanoate': '3-Hydroxypropanoate',
                         '(S)-Propane-1,2-diol': '(S)-Propanediol',
                         '(R)-Propane-1,2-diol': '(R)-Propanediol',
                         'D-Alanine': 'D-Alanine',
                         'Glycerol': 'Glycerol',
                         '(R)-Glycerate': '(R)-Glycerate',
                         'D-Glyceraldehyde': 'Glyceraldehyde',
                         'L-Lactate': 'L-Lactate',
                         'Dihydroxyacetone': 'Dihydroxyacetone',
                         'L-Alanine': 'L-Alanine',
                         'D-Lactate': 'D-Lactate',
                         'L-Serine': 'L-Serine',
                         'Pyruvate': 'Pyruvate',
                         'Ethanolamine': 'Ethanolamine',
                         'Ethanol': 'Ethanol',
                         'Acetaldehyde': 'Acetaldehyde',
                         'Glycine': 'Glycine',
                         'Acetate': 'Acetate',
                         'Glycolate C2H3O3': 'Glycolate',
                         'Urea CH4N2O': 'Urea',
                         'Formate': 'Formate',
                         'Reduced glutathione': 'Glutathione',
                         '5-Methylthio-D-ribose': '5-Methylthioribose',
                         '1,5-Diaminopentane': 'Cadaverine'}

metabolite_lookup_dict = {metabolite_names_dict[key]:key for key in metabolite_names_dict}

carbon_dict = {metabolite.name:str(metabolite.elements['C'])
               if 'C' in metabolite.elements else '0'
               for metabolite in model.metabolites}

carbon_dict = {metabolite:carbon_dict[metabolite] 
               if int(carbon_dict[metabolite])<=6 else '>6'
               for metabolite in metabolite_names_dict}
data = {'names':[metabolite_names_dict[product] for product in carbon_dict.keys()], 'number':list(carbon_dict.values())}
metabolite_df = pd.DataFrame(data,index=list(carbon_dict.keys()))
metabolite_order = metabolite_df.sort_values(by=['number','names']).names.values
uptake_dict = {'linear': linear_uptake, 'logistic': logistic_uptake}
print(carbon_dict)

{'Spermidine': '>6', 'L-Tryptophan': '>6', 'Thymidine C10H14N2O5': '>6', 'L-Phenylalanine': '>6', 'Adenosine': '>6', 'Indole': '>6', 'L-Tyrosine': '>6', 'Inosine': '>6', 'Xanthosine': '>6', 'Cytidine': '>6', 'Uridine': '>6', 'Quinate': '>6', 'L-Isoleucine': '6', 'L-Lysine': '6', 'Hexanoate (n-C6:0)': '6', 'L-Leucine': '6', 'L-Arginine': '6', 'L-Histidine': '6', 'D-Gluconate': '6', 'L-Idonate': '6', '5-Dehydro-D-gluconate': '6', 'Citrate': '6', 'Agmatine': '5', 'Ornithine': '5', 'L-Proline': '5', 'L-Valine': '5', 'Thymine C5H6N2O2': '5', 'Adenine': '5', 'Guanine': '5', 'Hypoxanthine': '5', 'O-Acetyl-L-serine': '5', 'Xanthine': '5', 'L-Glutamate': '5', '2-Oxoglutarate': '5', 'Putrescine': '4', 'L-Threonine': '4', '4-Aminobutanoate': '4', 'L-Homoserine': '4', 'Allantoin': '4', 'Succinate': '4', 'Uracil': '4', 'L-Asparagine': '4', 'L-Malate': '4', 'L-Aspartate': '4', 'L-Cysteine': '3', 'Glycerol 3-phosphate': '3', '3-Hydroxypropanoate': '3', '(S)-Propane-1,2-diol': '3', '(R)-Propane-1,2-di

In [29]:
import sys
import colorlover as cl
from plotly import tools, subplots
import plotly.graph_objs as go
import pickle
import plotly.io as pio
pio.templates.default = "none"
import os

try:
    _ = __IPYTHON__
except NameError:
    from plotly.offline import plot
else:
    if 'ipykernel' in sys.modules:
        from plotly.offline import init_notebook_mode
        from plotly.offline import iplot as plot
        from IPython.display import HTML
        HTML("""
             <script>
              var waitForPlotly = setInterval( function() {
              if( typeof(window.Plotly) !== "undefined" ){
              MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
              MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
              clearInterval(waitForPlotly);}}, 250 );
            </script>
            """
        )
        init_notebook_mode(connected=True)
    elif 'IPython' in sys.modules:
        from plotly.offline import plot
    else:
        warn('Unknown ipython configuration')
        from plotly.offline import plot


In [44]:
def stage_two_phenotype_calculator (pecaso_object, template_model):
    
    if pecaso_object.settings.uptake_fun in uptake_dict.keys():
        uptake_fun = uptake_dict[pecaso_object.settings.uptake_fun]
    else:
        raise KeyError('Unknown uptake function')
    
    biomass_rxn_id = pecaso_object.biomass_rxn.id
    substrate_rxn_id = pecaso_object.substrate_rxn.id
    target_rxn_id = pecaso_object.target_rxn.id

    model = deepcopy(template_model)

    if pecaso_object.two_stage_best_batch:
        
        biomass_flux = pecaso_object.two_stage_best_batch.stage_two_factor / 100 * \
                           max(pecaso_object.production_envelope.growth_rates)
        substrate_flux = -np.around(uptake_fun(biomass_flux, **pecaso_object.settings.uptake_params)+0.0000005,
                                    decimals=6)

        with model:
            model.reactions.get_by_id(biomass_rxn_id).bounds = (biomass_flux, biomass_flux)
            model.reactions.get_by_id(substrate_rxn_id).bounds = (substrate_flux, 1000)
            model.objective = target_rxn_id
            target_flux = model.optimize().objective_value

        fluxes = [biomass_flux, substrate_flux, target_flux]
        return {metabolite_names_dict[model.reactions.get_by_id(target_rxn_id).reactants[0].name]:fluxes}
    else:
        return {metabolite_names_dict[model.reactions.get_by_id(target_rxn_id).reactants[0].name]:None}

In [6]:
pickling_off = open("pecaso_obj_prod.pickle","rb")
pecaso_dict_prod = pickle.load(pickling_off)
pickling_off.close()

pickling_off = open("pecaso_obj_yield_prod_constrained_2_native_mets.pickle","rb")
pecaso_dict_yield_2_prod = pickle.load(pickling_off)
pickling_off.close()

pickling_off = open("pecaso_obj_prod_yield_constrained_75_native_mets.pickle","rb")
pecaso_dict_prod_75_yield = pickle.load(pickling_off)
pickling_off.close()

In [31]:
unconstrained_prod_obj = {'stage_one_factor': [pecaso_dict_prod[product].two_stage_best_batch.stage_one_factor
                                               if pecaso_dict_prod[product].two_stage_best_batch else np.nan
                                               for product in pecaso_dict_prod.keys()],
                          'stage_two_factor': [pecaso_dict_prod[product].two_stage_best_batch.stage_two_factor 
                                               if pecaso_dict_prod[product].two_stage_best_batch else np.nan
                                               for product in pecaso_dict_prod.keys()]}

prod_obj_75_constr_yield = {'stage_one_factor': [pecaso_dict_prod_75_yield[product].two_stage_best_batch.stage_one_factor
                                                 if pecaso_dict_prod_75_yield[product].two_stage_best_batch else np.nan 
                                                 for product in pecaso_dict_prod_75_yield.keys()],
                            'stage_two_factor': [pecaso_dict_prod_75_yield[product].two_stage_best_batch.stage_two_factor
                                                 if pecaso_dict_prod_75_yield[product].two_stage_best_batch else np.nan
                                                 for product in pecaso_dict_prod_75_yield.keys()]}

yield_obj_2_constr_prod = {'stage_one_factor': [pecaso_dict_yield_2_prod[product].two_stage_best_batch.stage_one_factor
                                                if pecaso_dict_yield_2_prod[product].two_stage_best_batch else np.nan
                                                for product in pecaso_dict_yield_2_prod.keys()],
                           'stage_two_factor': [pecaso_dict_yield_2_prod[product].two_stage_best_batch.stage_two_factor 
                                                if pecaso_dict_yield_2_prod[product].two_stage_best_batch else np.nan
                                                for product in pecaso_dict_yield_2_prod.keys()]}

unconstrained_prod_df = pd.DataFrame(unconstrained_prod_obj, index = [product for product in pecaso_dict_prod.keys()])
prod_obj_75_constr_yield_df = pd.DataFrame(prod_obj_75_constr_yield, index = [product for product in pecaso_dict_prod_75_yield.keys()])
yield_obj_2_constr_prod_df = pd.DataFrame(yield_obj_2_constr_prod, index = [product for product in pecaso_dict_yield_2_prod.keys()])


In [32]:
phenotype_df = pd.concat([unconstrained_prod_df,prod_obj_75_constr_yield_df,yield_obj_2_constr_prod_df], axis=1,
                          keys=['productivity','80_yield_constrained_productivity','2_productivity_constrained_yield'])
display(phenotype_df)

productivity                   ... 2_productivity_constrained_yield                 
                    stage_one_factor stage_two_factor  ...                 stage_one_factor stage_two_factor
Spermidine                     100.0        31.480168  ...                              NaN              NaN
L-Tryptophan                   100.0        33.162115  ...                       100.000000        31.941040
Thymidine                      100.0        32.956812  ...                       100.000000        16.688276
L-Phenylalanine                100.0        33.780872  ...                       100.000000        30.341561
Adenosine                      100.0        33.901782  ...                       100.000000        15.166868
Indole                         100.0        34.933787  ...                              NaN              NaN
L-Tyrosine                     100.0        34.294738  ...                       100.000000        22.213830
Inosine                        100.0        33.743137  ...                       100.000000        14.481314
Xanthosine                     100.0        33.775548  ...                       100.000000        18.386488
Cytidine                       100.0        33.754916  ...                       100.000000        14.089105
Uridine                        100.0        33.950424  ...                       100.000000        14.162089
Quinate                        100.0        33.997675  ...                       100.000000        14.403061
L-Isoleucine                   100.0        34.404097  ...                       100.000000        30.693411
L-Lysine                       100.0        34.782994  ...                       100.000000        20.146266
Hexanoate                      100.0        33.891702  ...                       100.000000        30.447185
L-Leucine                      100.0        35.085216  ...                       100.000000        21.438985
L-Arginine                     100.0        34.108581  ...                       100.000000        14.917536
L-Histidine                    100.0        34.229658  ...                       100.000000        15.928707
D-Gluconic Acid                100.0        34.008826  ...                       100.000000        10.985374
L-Idonic Acid                  100.0        34.277020  ...                       100.000000        11.140911
5-Ketogluconate                100.0        34.196700  ...                       100.000000        19.718195
Citric Acid                    100.0        34.067485  ...                       100.000000         8.883823
Agmatine                       100.0        34.438213  ...                       100.000000        23.346000
Ornithine                      100.0        34.368906  ...                       100.000000        16.999050
L-Proline                      100.0        33.409465  ...                       100.000000        22.193386
L-Valine                       100.0        34.142865  ...                       100.000000        18.380778
Thymine                        100.0        33.285696  ...                       100.000000        18.048325
Adenine                        100.0        34.005295  ...                       100.000000        14.752342
Guanine                        100.0        34.261685  ...                       100.000000        12.950757
Hypoxanthine                   100.0        34.535920  ...                       100.000000        14.066901
O-Acetyl-L-serine              100.0        34.817961  ...                       100.000000        12.061373
Xanthine                       100.0        34.946335  ...                       100.000000        11.588443
L-Glutamate                    100.0        33.956797  ...                       100.000000        11.803892
α-Ketoglutarate                100.0        33.668059  ...                       100.000000        10.768362
Putrescine                     100.0        35.143433  ...                       100.000000        3

In [39]:
phenotype_dict_list = Parallel(n_jobs=num_cores, verbose=5)(delayed(stage_two_phenotype_calculator)
                              (pecaso_object, model)
                              for pecaso_object in pecaso_dict_prod.values())

phenotype_dict = {list(phenotype_dict.keys())[0]: list(phenotype_dict.values())[0] 
                  for phenotype_dict in phenotype_dict_list}

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   19.8s
C:\Users\kraj5\Anaconda3\envs\dyssco_plotly_4\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

[Parallel(n_jobs=8)]: Done  70 out of  70 | elapsed:  6.1min remaining:    0.0s
[Parallel(n_jobs=8)]: Done  70 out of  70 | elapsed:  6.1min finished


In [41]:
phenotype_dict_list = Parallel(n_jobs=num_cores, verbose=5)(delayed(stage_two_phenotype_calculator)
                              (pecaso_object, model)
                              for pecaso_object in pecaso_dict_prod_75_yield.values())

phenotype_dict_prod_75_yield = {list(phenotype_dict.keys())[0]: list(phenotype_dict.values())[0] 
                                for phenotype_dict in phenotype_dict_list}

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   20.4s
C:\Users\kraj5\Anaconda3\envs\dyssco_plotly_4\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

[Parallel(n_jobs=8)]: Done  70 out of  70 | elapsed:  5.9min remaining:    0.0s
[Parallel(n_jobs=8)]: Done  70 out of  70 | elapsed:  5.9min finished


In [45]:
phenotype_dict_list = Parallel(n_jobs=num_cores, verbose=5)(delayed(stage_two_phenotype_calculator)
                              (pecaso_object, model)
                              for pecaso_object in pecaso_dict_yield_2_prod.values())

phenotype_dict_yield_2_prod = {list(phenotype_dict.keys())[0]: list(phenotype_dict.values())[0] 
                                for phenotype_dict in phenotype_dict_list}

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
C:\Users\kraj5\Anaconda3\envs\dyssco_plotly_4\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   13.8s
[Parallel(n_jobs=8)]: Done  70 out of  70 | elapsed:  6.0min remaining:    0.0s
[Parallel(n_jobs=8)]: Done  70 out of  70 | elapsed:  6.0min finished


In [46]:
flux_dicts = {'productivity': phenotype_dict,
              'productivity_75_yield': phenotype_dict_prod_75_yield,
              'yield_2_productivity': phenotype_dict_yield_2_prod}

pickling_on = open("flux_dicts.pickle","wb")
pickle.dump(flux_dicts, pickling_on)
pickling_on.close() 

In [49]:
flux_dicts['productivity_75_yield']

{'Spermidine': [0.16541426347624258, -4.218972, 0.68616841085536],
 'L-Tryptophan': [0.1708215234926666, -4.327134, 1.1626865071852914],
 'Thymidine': [0.16769391604749173, -4.264714, 1.3356664672361773],
 'L-Phenylalanine': [0.1744319731011909, -4.398704, 1.4064192910327495],
 'Adenosine': [0.16957848147806454, -4.302372, 1.3861914569599363],
 'Indole': [0.1742122170810217, -4.394363, 1.4432955339667146],
 'L-Tyrosine': [0.17940557504764823, -4.496429, 1.4930400452659462],
 'Inosine': [0.17687558389688685, -4.446844, 1.4789669802907919],
 'Xanthosine': [0.1810739395435291, -4.528983, 1.567795432852678],
 'Cytidine': [0.16432784873905956, -4.1971, 1.5659760528320334],
 'Uridine': [0.16522611984309707, -4.215187, 1.671896535712529],
 'Quinate': [0.17590426692325284, -4.427738, 2.076328889838145],
 'L-Isoleucine': [0.1653040322259553, -4.216755, 1.8417656776176408],
 'L-Lysine': [0.17819762656403018, -4.472787, 2.036115121428457],
 'Hexanoate': [0.16885178659787065, -4.287868, 1.98756593